In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import matplotlib.pyplot as plt    
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import os
import re
import csv

init_notebook_mode(connected=True)

In [2]:
#vs F. vesca genome
e = "S1_ccs_3_99.fasta_vs_Fragaria_vesca_v1.1.a2_cds_removed.fasta_nucl.db"
#vs selected NBS-LRR genes
f = "S1_ccs_3_99.fasta_vs_vesca_v1.1_nblrrs_augustus_cds_nucl.db"
#Output from NBS Parser
nbs_pars = "S1_ccs_3_99_nlr.tsv"

#Lengths
i = "S1_ccs_3_99_lengths.txt"
g = "Fragaria_vesca_v1.1.a2_cds_lengths.txt"
h = "vesca_v1.1_nblrrs_augustus_cds_lengths.txt"

df = pd.read_table(e, sep="\t", header=0,  index_col=False,
    names=["qseqid", "sseqid", "pident", "aln_length", "mismatch", 
           "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qlen", "slen", "sstrand"])
df2 = pd.read_table(f, sep="\t", header=0,  index_col=False,
    names=["qseqid", "sseqid", "pident", "aln_length", "mismatch", 
           "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qlen", "slen", "sstrand"])

all_sequences = pd.read_table(i, sep=",", header=0,  index_col=False,
    names=["seqid", "length"])

all_vesca = pd.read_table(g, sep=",", header=0,  index_col=False,
    names=["vescaid", "length"])
all_nblrrs = pd.read_table(h, sep=",", header=0,  index_col=False,
    names=["nblrrid", "length"])
all_nbs_parser = pd.read_table(nbs_pars, sep="\t", header=0,  index_col=False,
    names=["contig", "pred_id", "status", "start", "end", "strand", "domains"])

In [3]:
#Homologs of how many vesca genes from original bait design are present in the Ren-seq output?
union1 = pd.Series(list(set(df2['sseqid']).intersection(set(all_nblrrs['nblrrid']))))
len(union1)

322

In [4]:
#Homologs of how many vesca genes from original bait design are present in the Ren-seq output? (Max 90% divergence)
df2_max90 = df2[df2['pident'] >= 90]
union90 = pd.Series(list(set(df2_max90['sseqid']).intersection(set(all_nblrrs['nblrrid']))))
len(union90)

318

In [5]:
#How many vesca genes from original bait design are NOT present in the Ren-seq output?
diff2 = pd.Series(list(set(all_nblrrs['nblrrid']).difference(set(df2['sseqid']))))
len(diff2)

6

In [6]:
#How many input sequences have a hit to the vesca genes in the original bait design?
union2 = pd.Series(list(set(df2['qseqid']).intersection(set(all_sequences['seqid']))))
len(union2)

20054

In [7]:
###Are any additional vesca genes present? 
#All the hits in all the genes, max. 90 divergence:
df_max90 = df[df['pident'] >= 90]
union9 = pd.Series(list(set(df_max90['sseqid']).intersection(set(all_vesca['vescaid']))))
len(union9)

4294

In [8]:
#How many input sequences have a hit on the vesca genome?
union_v = pd.Series(list(set(df['qseqid']).intersection(set(all_sequences['seqid']))))
len(union_v)

22118

In [9]:
#How many input sequences DO NOT have a hit on the vesca genome?
diff3 = pd.Series(list(set(all_sequences['seqid']).difference(set(df['qseqid']))))
len(diff3)

758

In [10]:
#How many contigs contain a predicted NBS-LRR gene (according to NBS Parser?
just_ids = (all_nbs_parser.iloc[:,[0,2]])
just_ids_dedup = just_ids.drop_duplicates()
len(just_ids_dedup)

8637

In [11]:
#How many of them complete?
complete = just_ids[just_ids['status'] == "complete"]
len(complete)

1870

In [12]:
#How many of them partial?
partial = just_ids[just_ids['status'] == "partial"]
len(partial)

3340

In [13]:
#How many of them pseudogenes?
pseudogene = just_ids[just_ids['status'] == "pseudogene"]
len(pseudogene)

3467

In [14]:
###Investigate the distribution of coding, 5' upstream and 3' upstream regions among all the hits in the vesca genome.

In [15]:
#Prefilter to retain only top hit (in terms of %aln) for each query sequence.
#df.groupby(['qseqid'], sort=False)['pident'].max()
idx = df.groupby(['qseqid'])['aln_length'].transform(max) == df['aln_length']
df_top = df[idx]
select_matches = (df_top.iloc[:,[0,1]])
select_dict = select_matches.set_index('qseqid')['sseqid'].to_dict()
lst_index = list()
for index, row in select_matches.iterrows():
    if row['qseqid'] in select_dict:
        if select_dict[row['qseqid']] == row['sseqid']:
            lst_index.append(index)
df_prefilt = (df.ix[lst_index])
all_alignment_lengths = list()
all_relative_alignment_lengths = list()
all_five_lengths = list()
all_three_lengths = list()
for key in select_dict.keys():
    aln_length = 0
    UTR_5_len = 0
    UTR_3_len = 0
    df_slice = df_prefilt[df_prefilt['qseqid'] == key]
    for i, r in df_slice.iterrows():
        aln_length += r['aln_length']
        #Check if matches start position
        if r['sstart'] == 1:
            UTR_5_len = r['qlen'] - r['qstart']
            all_five_lengths.append(UTR_5_len)
        elif r['send'] == 1:
            UTR_5_len = r['qlen'] - r['qstart'] 
            all_five_lengths.append(UTR_5_len)
        #Check if matches end position
        if r['sstart'] >= r['slen'] - 3:
            UTR_3_len = r['qlen'] - r ['qend']
            all_three_lengths.append(UTR_3_len)
        elif r['send'] >= r['slen'] - 3:
            UTR_3_len = r['qlen'] - r ['qend']
            all_three_lengths.append(UTR_3_len)
    all_alignment_lengths.append(aln_length)
    relative_length = float(aln_length)/float(r['slen'])
    all_relative_alignment_lengths.append(relative_length)

In [16]:
#Histogram for CDS total length
cds = pd.Series(all_alignment_lengths).to_frame(name="Frequency")
cds.iplot(kind='histogram', bins=100, color="black")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:334: SNIMissingWarning:

An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



In [17]:
#Histogram for CDS relative length (fraction reference)
#cds_rel = pd.Series(all_relative_alignment_lengths).value_counts()
cds_rel = pd.Series(all_relative_alignment_lengths).to_frame(name="Frequency")
cds_rel.iplot(kind='histogram', bins=50, color="pink")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



In [18]:
#Histogram for 5' UTR lengths
five_l = pd.Series(all_five_lengths).to_frame(name="Frequency")
five_l.iplot(kind='histogram', bins=100, color="green")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



In [22]:
#Histogram for 3' UTR lengths
three_l = pd.Series(all_three_lengths).to_frame(name="Frequency")
three_l.iplot(kind='histogram', bins=100, color="purple")

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



PlotlyRequestError: Hey there! You've hit one of our API request limits. 

To get unlimited API calls(10,000/day), please upgrade to a paid plan. 

UPGRADE HERE: https://goo.gl/i7glmM 

Thanks for using Plotly! Happy Plotting!